# Lab 14 - Part 1: Production Infrastructure Setup

## Overview
This notebook covers the setup of production infrastructure for Neo4j deployment including:
- Environment configuration (Dev, Staging, Prod)
- Production configuration management
- Infrastructure setup
- Dependency verification

**Duration:** 10 minutes  
**Prerequisites:** Neo4j driver installed

## Install Production Dependencies

First, we'll install all required packages for production deployment with proper error handling.

In [ ]:
# Install production deployment dependencies
import subprocess
import sys
import os
from datetime import datetime, timedelta
import json
import logging

def install_package(package_name):
    """Install package with error handling"""
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", package_name], 
                            capture_output=True, text=True)
        print(f"✓ {package_name} installed successfully")
        return True
    except subprocess.CalledProcessError as e:
        print(f"✗ Failed to install {package_name}: {e}")
        return False
    except Exception as e:
        print(f"✗ Error installing {package_name}: {e}")
        return False

# Production deployment packages
packages = [
    "docker==6.1.3",
    "kubernetes==27.2.0", 
    "prometheus-client==0.17.1",
    "psutil==5.9.5",
    "cryptography==41.0.4",
    "pyjwt==2.8.0",
    "schedule==1.2.0",
    "paramiko==3.3.1"
]

# Install packages
print("📦 Installing production deployment dependencies...")
successful_installs = 0
for package in packages:
    if install_package(package):
        successful_installs += 1

print(f"\n✓ Production dependencies installed: {successful_installs}/{len(packages)} successful")

if successful_installs < len(packages):
    print("⚠️ Some packages failed to install - continuing with available dependencies")
else:
    print("✅ All production deployment dependencies installed successfully")

## Import Production Libraries

Import all required libraries with fallback handling for missing dependencies.

In [ ]:
# Import production libraries with error handling
print("📚 Importing production libraries...")

# Standard library imports
import time
import hashlib
import secrets
import threading
import subprocess
import shutil
import json
import logging
from datetime import datetime, timedelta

# Try importing optional dependencies with fallbacks
try:
    import docker
    DOCKER_AVAILABLE = True
    print("✓ Docker library imported")
except ImportError:
    DOCKER_AVAILABLE = False
    print("⚠️ Docker library not available - container operations will be simulated")

try:
    import psutil
    PSUTIL_AVAILABLE = True
    print("✓ Psutil library imported")
except ImportError:
    PSUTIL_AVAILABLE = False
    print("⚠️ Psutil not available - system metrics will be simulated")

try:
    import jwt
    JWT_AVAILABLE = True
    print("✓ JWT library imported")
except ImportError:
    JWT_AVAILABLE = False
    print("⚠️ JWT not available - using basic token generation")

try:
    from cryptography.fernet import Fernet
    CRYPTOGRAPHY_AVAILABLE = True
    print("✓ Cryptography library imported")
except ImportError:
    CRYPTOGRAPHY_AVAILABLE = False
    print("⚠️ Cryptography not available - using basic encryption")

try:
    from prometheus_client import Counter, Histogram, Gauge, start_http_server
    PROMETHEUS_AVAILABLE = True
    print("✓ Prometheus client imported")
except ImportError:
    PROMETHEUS_AVAILABLE = False
    print("⚠️ Prometheus client not available - metrics will be simulated")

try:
    import schedule
    SCHEDULE_AVAILABLE = True
    print("✓ Schedule library imported")
except ImportError:
    SCHEDULE_AVAILABLE = False
    print("⚠️ Schedule not available - using basic scheduling")

try:
    import paramiko
    PARAMIKO_AVAILABLE = True
    print("✓ Paramiko library imported")
except ImportError:
    PARAMIKO_AVAILABLE = False
    print("⚠️ Paramiko not available - remote operations will be simulated")

# Neo4j import (should be available from previous labs)
try:
    from neo4j import GraphDatabase
    NEO4J_AVAILABLE = True
    print("✓ Neo4j driver imported")
except ImportError:
    NEO4J_AVAILABLE = False
    print("❌ Neo4j driver not available - this is required for the lab")
    raise ImportError("Neo4j driver is required - please install with: pip install neo4j")

print("✅ Production libraries imported successfully (with available dependencies)")

## Production Configuration Management

Create a configuration manager to handle multiple environments (development, staging, production).

In [ ]:
class ProductionConfig:
    """Production environment configuration management"""
    
    def __init__(self):
        self.environments = {
            "development": {
                "neo4j_uri": "bolt://localhost:7687",
                "neo4j_user": "neo4j",
                "neo4j_password": "password",
                "security_level": "basic",
                "monitoring_enabled": False,
                "backup_enabled": False
            },
            "staging": {
                "neo4j_uri": "bolt://staging-neo4j:7687",
                "neo4j_user": "neo4j_staging",
                "neo4j_password": "staging_secure_password_123",
                "security_level": "enhanced",
                "monitoring_enabled": True,
                "backup_enabled": True
            },
            "production": {
                "neo4j_uri": "bolt://prod-neo4j-cluster:7687",
                "neo4j_user": "neo4j_prod",
                "neo4j_password": "prod_ultra_secure_password_456",
                "security_level": "maximum",
                "monitoring_enabled": True,
                "backup_enabled": True,
                "encryption_enabled": True,
                "audit_logging": True,
                "high_availability": True
            }
        }
        
        self.current_environment = "production"
        self.deployment_id = f"deploy_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
    
    def get_config(self, environment=None):
        """Get configuration for specified environment"""
        env = environment or self.current_environment
        return self.environments.get(env, {})
    
    def validate_environment(self, environment):
        """Validate environment configuration"""
        config = self.get_config(environment)
        
        required_fields = ["neo4j_uri", "neo4j_user", "neo4j_password"]
        missing_fields = [field for field in required_fields if not config.get(field)]
        
        if missing_fields:
            raise ValueError(f"Missing required configuration: {missing_fields}")
        
        return True

# Initialize production configuration
prod_config = ProductionConfig()
print("✓ Production configuration initialized")

# Display configuration for all environments
print("\n📋 Environment Configurations:")
for env_name in ["development", "staging", "production"]:
    config = prod_config.get_config(env_name)
    print(f"\n{env_name.upper()}:")
    print(f"  - URI: {config['neo4j_uri']}")
    print(f"  - Security Level: {config['security_level']}")
    print(f"  - Monitoring: {config['monitoring_enabled']}")
    print(f"  - Backup: {config['backup_enabled']}")

## Verify Environment Configuration

Test the configuration for all environments to ensure they're properly set up.

In [ ]:
print("🔍 Validating Environment Configurations...\n")

for env_name in ["development", "staging", "production"]:
    try:
        prod_config.validate_environment(env_name)
        print(f"✓ {env_name.upper()} configuration is valid")
    except ValueError as e:
        print(f"✗ {env_name.upper()} configuration error: {e}")

print(f"\n✅ All environments configured successfully")
print(f"🆔 Deployment ID: {prod_config.deployment_id}")

## Key Takeaways

In this notebook, you've:
1. ✅ Installed production deployment dependencies
2. ✅ Imported required libraries with fallback handling
3. ✅ Created multi-environment configuration management
4. ✅ Validated configuration for Dev, Staging, and Production

**Next Steps:** Proceed to notebook 02 for Security and Authentication implementation.